In [2]:
import pandas as pd 
import json
import os

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [4]:
input_dir = os.getenv('TRIVY_SCAN_REPORTS_DIR', default = '../../.bob/va-reports/trivy')
output_dir = os.getenv('TRIVY_ANALYSIS_REPORTS_DIR', default ='../../.bob/va-reports/analysis-reports/trivy')


In [5]:
from datetime import datetime
runDate= datetime.today().strftime('%Y%m%d')
runDate

'20220428'

#### Report to Name Mapping

In [6]:
REPORT_NAMES={  'CVE_COUNT_BY_SEVERITY' : 'cve_count_by_severity',
                'CVE_COUNT_BY_SEVERITY_AND_AGE' : 'cve_count_by_severity_and_age',
                'IMAGE_COUNT_BY_CVE': 'image_count_by_cve',
                'DETAILED_REPORT': 'detailed_report', 
                'CONSOLIDATED_EXCEL_REPORT' : f'mxe_va_analysis_report_{runDate}'
}

## Get all 2PP, 3PP scan json reports by doing a recursive walk of scan output folder

In [7]:
json_files = [os.path.join(dp, f) 
            for dp, dn, filenames in os.walk(input_dir) 
                for f in filenames if f.endswith('.json')]
print(len(json_files))
json_files

66


['../../.bob/va-reports/trivy/internal-reports/keycloak-init.json',
 '../../.bob/va-reports/trivy/internal-reports/db-init.json',
 '../../.bob/va-reports/trivy/internal-reports/pre-upgrade-job.json',
 '../../.bob/va-reports/trivy/internal-reports/gui.json',
 '../../.bob/va-reports/trivy/internal-reports/model-service.json',
 '../../.bob/va-reports/trivy/internal-reports/jupyterlab.json',
 '../../.bob/va-reports/trivy/internal-reports/eric-mxe-gatekeeper.json',
 '../../.bob/va-reports/trivy/internal-reports/author-service.json',
 '../../.bob/va-reports/trivy/internal-reports/mxe-deployer-init.json',
 '../../.bob/va-reports/trivy/internal-reports/mxe-deployer-service.json',
 '../../.bob/va-reports/trivy/internal-reports/pypi-eea.json',
 '../../.bob/va-reports/trivy/internal-reports/model-packager.json',
 '../../.bob/va-reports/trivy/internal-reports/default-backend.json',
 '../../.bob/va-reports/trivy/internal-reports/kafka-pyconsumer.json',
 '../../.bob/va-reports/trivy/internal-reports

### Load trivy report scans

Considerations

1. Some files only have "null" as content.  So on reading file content as json, json_data will not contain anything, hence; only proceed loading the json_data into pd if it contains anything


2. Each file contains a list of dicts. And we are interested in the Vulnerabilities key inside every dict.
pandas json_normalize can be used to flatten nested dicts present in Vulnerabilities section , however not every dict in the list has Vulnerabilities key and this is breaking the load to pandas. To circumvent this issue, preprocess the json_data and insert a key 'Vulnerabilities' if it does not exist in the dict
Ex: 
We explicitly look for 'Vulnerabilities' key to normalize, but entries like below dont have this key. So we insert `'Vulnerabilities' :[]`
```
{
    "Target": "opt/spark/jars/commons-beanutils-1.9.4.jar",
    "Class": "lang-pkgs",
    "Type": "jar"
}
```

In [8]:
def load_json(json_file):
    with open(json_file) as jsonReport:
        json_data= json.load(jsonReport)
    if json_data:
        for entry in json_data: 
            if not 'Vulnerabilities' in entry: 
                entry['Vulnerabilities']=[]
    return json_data

def load_json_normalized(json_file):
    json_data=load_json(json_file)
    if json_data:
        df=pd.json_normalize(json_data, record_path=['Vulnerabilities'], meta=['Target','Class', 'Type'])
        df.insert(loc=0, column='image_name', value=os.path.basename(json_file))
        return df
    return pd.DataFrame() 

def load_schema_version2_json(json_file):
    with open(json_file) as jsonReport:
        json_data= json.load(jsonReport)
    if json_data:
        if "Results" in json_data: 
            results = json_data['Results']
            for entry in results: 
                if not 'Vulnerabilities' in entry: 
                    entry['Vulnerabilities']=[]
                if not 'Type' in entry:
                    entry['Type']="" 
    return json_data   

def load_schema_version2_json_normalized(json_file):
    json_data=load_schema_version2_json(json_file)
    print(json_file)
    if json_data and 'Results' in json_data:
        artifactName=json_data['ArtifactName']
        artifactType=json_data['ArtifactType']
        metadata=json_data['Metadata']
        metadataOSFamily='Unknown'
        metadataOSName='Unknown'
        repoTags='Unknown'
        repoDigests='Unknown'
        if 'OS' in  metadata: 
            metadataOSFamily=metadata['OS']['Family']
            metadataOSName=metadata['OS']['Name']
        if 'RepoTags' in metadata:
            repoTags=metadata['RepoTags'][0]
        if 'RepoDigests' in metadata:
            repoDigests=metadata['RepoDigests'][0]
        df=pd.json_normalize(json_data['Results'], record_path=['Vulnerabilities'], meta=['Target', 'Class', 'Type'])
        df.insert(loc=0, column='image_name', value=os.path.basename(json_file))
        df.insert(loc=0, column='artifactName', value=os.path.basename(artifactName))
        df.insert(loc=0, column='artifactType', value=os.path.basename(artifactType))
        df.insert(loc=0, column='metadataOSFamily', value=os.path.basename(metadataOSFamily))
        df.insert(loc=0, column='metadataOSName', value=os.path.basename(metadataOSName))
        df.insert(loc=0, column='repoTags', value=os.path.basename(repoTags))
        df.insert(loc=0, column='repoDigests', value=os.path.basename(repoDigests))
        return df
    return pd.DataFrame() 

dfs=[load_schema_version2_json_normalized(json_file) for json_file in json_files]
df=pd.concat(dfs)

../../.bob/va-reports/trivy/internal-reports/keycloak-init.json
../../.bob/va-reports/trivy/internal-reports/db-init.json
../../.bob/va-reports/trivy/internal-reports/pre-upgrade-job.json
../../.bob/va-reports/trivy/internal-reports/gui.json
../../.bob/va-reports/trivy/internal-reports/model-service.json
../../.bob/va-reports/trivy/internal-reports/jupyterlab.json
../../.bob/va-reports/trivy/internal-reports/eric-mxe-gatekeeper.json
../../.bob/va-reports/trivy/internal-reports/author-service.json
../../.bob/va-reports/trivy/internal-reports/mxe-deployer-init.json
../../.bob/va-reports/trivy/internal-reports/mxe-deployer-service.json
../../.bob/va-reports/trivy/internal-reports/pypi-eea.json
../../.bob/va-reports/trivy/internal-reports/model-packager.json
../../.bob/va-reports/trivy/internal-reports/default-backend.json
../../.bob/va-reports/trivy/internal-reports/kafka-pyconsumer.json
../../.bob/va-reports/trivy/internal-reports/model-python-base.json
../../.bob/va-reports/trivy/intern

## Distinct Vuln Sources

AFAIK only NVD data (VulnerabilityIDs beginning with CVE) would have the Published/LastModified dates. Should we just ignore the rest?

In [9]:
vulnDF =df[['VulnerabilityID']].drop_duplicates()

vulnDF['source']=vulnDF.apply(lambda x: x['VulnerabilityID'].split("-")[0], axis=1)

vulnDF['source'].value_counts()



CVE     292
GMS      11
SUSE     10
GHSA      1
Name: source, dtype: int64

## Counts by Severity

Reused code written for Anchore Scan, just edited the column names to match trivy dataframe column names

In [10]:
from collections import defaultdict
SEVERITY_DICT = defaultdict(lambda: 7)
SEVERITY_DICT.update({"CRITICAL":1, "HIGH":2, "MEDIUM":3, "LOW":4, "NEGLIGIBLE":5,"UNKNOWN":6})

In [11]:
unique_cves = df[['Severity', 'VulnerabilityID']].drop_duplicates()
severity_counts_df = pd.DataFrame(unique_cves['Severity'].value_counts())
severity_counts_df=severity_counts_df.reset_index()
severity_counts_df.columns = ['Severity', 'CVE_Count']
severity_counts_df['Rank'] = severity_counts_df['Severity'].map(SEVERITY_DICT)
severity_counts_df.sort_values(by=['Rank','Severity', 'CVE_Count'],inplace=True)
severity_counts_df = severity_counts_df.drop(labels=['Rank'],axis=1).reset_index(drop=True)
severity_counts_df

,Severity,CVE_Count
0,CRITICAL,6
1,HIGH,50
2,MEDIUM,154
3,LOW,98
4,UNKNOWN,12


In [12]:
severity_counts_df.to_html(f"{output_dir}/{REPORT_NAMES['CVE_COUNT_BY_SEVERITY']}.html")
severity_counts_df.to_csv(f"{output_dir}/{REPORT_NAMES['CVE_COUNT_BY_SEVERITY']}.csv", index=False)

In [14]:
detailed_report_df = df[['repoTags', 'Type', 'Class', 'VulnerabilityID', 'Severity', 'PublishedDate', 'LastModifiedDate','Description']].drop_duplicates()
detailed_report_df['Rank'] = detailed_report_df['Severity'].map(SEVERITY_DICT)
detailed_report_df.rename(columns={'repoTags': 'ImageName'}, inplace=True)

detailed_report_df.sort_values(by=['Rank', 'ImageName'], ascending=[True, True],inplace=True)
detailed_report_df = detailed_report_df.drop(labels=['Rank'],axis=1).reset_index(drop=True)
detailed_report_df.head()

,ImageName,Type,Class,VulnerabilityID,Severity,PublishedDate,LastModifiedDate,Description
0,author-service:2.3.0-dev-51,jar,lang-pkgs,CVE-2022-0839,CRITICAL,2022-03-04T15:15:00Z,2022-03-16T14:06:00Z,Improper Restriction of XML External Entity Re...
1,eric-ctrl-bro:5.4.0-35,jar,lang-pkgs,CVE-2022-22965,CRITICAL,2022-04-01T23:15:00Z,2022-04-21T14:40:00Z,A Spring MVC or Spring WebFlux application run...
2,eric-pm-server:9.0.0-27,gobinary,lang-pkgs,CVE-2021-43816,CRITICAL,2022-01-05T19:15:00Z,2022-04-01T14:50:00Z,containerd is an open source container runtime...
3,eric-sec-access-mgmt-image:11.0.0-73,jar,lang-pkgs,CVE-2022-0839,CRITICAL,2022-03-04T15:15:00Z,2022-03-16T14:06:00Z,Improper Restriction of XML External Entity Re...
4,eric-sec-access-mgmt-image:11.0.0-73,jar,lang-pkgs,CVE-2022-21724,CRITICAL,2022-02-02T12:15:00Z,2022-04-14T18:15:00Z,pgjdbc is the offical PostgreSQL JDBC Driver. ...


In [15]:
detailed_report_df.to_html(f"{output_dir}/{REPORT_NAMES['DETAILED_REPORT']}.html")
detailed_report_df.to_csv(f"{output_dir}/{REPORT_NAMES['DETAILED_REPORT']}.csv", index=False)

In [16]:
modified_base_df = df[df['Severity'].isin(['CRITICAL', 'HIGH'])]
modified_base_df.head(5)

,repoDigests,repoTags,metadataOSName,metadataOSFamily,artifactType,artifactName,image_name,VulnerabilityID,PkgName,InstalledVersion,FixedVersion,SeveritySource,Title,Description,Severity,References,Layer.DiffID,DataSource.ID,DataSource.Name,DataSource.URL,PkgPath,PrimaryURL,CweIDs,PublishedDate,LastModifiedDate,CVSS.nvd.V2Vector,CVSS.nvd.V3Vector,CVSS.nvd.V2Score,CVSS.nvd.V3Score,CVSS.redhat.V3Vector,CVSS.redhat.V3Score,Target,Class,Type,CVSS.redhat.V2Vector,CVSS.redhat.V2Score
0,keycloak-init@sha256:a616d074b14f04fd4a9aebd8b...,keycloak-init:2.3.0-dev-51,15.3,suse linux enterprise server,container_image,keycloak-init:2.3.0-dev-51,keycloak-init.json,SUSE-SU-2022:0844-1,libexpat1,2.2.5-3.15.1,2.2.5-3.19.1,suse-cvrf,Security update for expat,This update for expat fixes the following issu...,HIGH,"[https://bugzilla.suse.com/1196025, https://bu...",sha256:a5712801d34708e0dde3b47ff22db82bd2701dc...,suse-cvrf,SUSE CVRF,https://ftp.suse.com/pub/projects/security/cvrf/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,os-pkgs,suse linux enterprise server,NaN,NaN
1,keycloak-init@sha256:a616d074b14f04fd4a9aebd8b...,keycloak-init:2.3.0-dev-51,15.3,suse linux enterprise server,container_image,keycloak-init:2.3.0-dev-51,keycloak-init.json,SUSE-SU-2022:1149-1,libfreebl3,3.68.2-3.64.2,3.68.3-150000.3.67.1,suse-cvrf,Security update for mozilla-nss,This update for mozilla-nss fixes the followin...,HIGH,"[https://bugzilla.suse.com/1197903, https://li...",sha256:04fcd3db169b7fe1dfcb69d49bf7aae5ef1de03...,suse-cvrf,SUSE CVRF,https://ftp.suse.com/pub/projects/security/cvrf/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,os-pkgs,suse linux enterprise server,NaN,NaN
2,keycloak-init@sha256:a616d074b14f04fd4a9aebd8b...,keycloak-init:2.3.0-dev-51,15.3,suse linux enterprise server,container_image,keycloak-init:2.3.0-dev-51,keycloak-init.json,SUSE-SU-2022:1158-1,liblzma5,5.2.3-4.3.1,5.2.3-150000.4.7.1,suse-cvrf,Security update for xz,This update for xz fixes the following issues:...,HIGH,"[https://bugzilla.suse.com/1198062, https://li...",sha256:a5712801d34708e0dde3b47ff22db82bd2701dc...,suse-cvrf,SUSE CVRF,https://ftp.suse.com/pub/projects/security/cvrf/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,os-pkgs,suse linux enterprise server,NaN,NaN
3,keycloak-init@sha256:a616d074b14f04fd4a9aebd8b...,keycloak-init:2.3.0-dev-51,15.3,suse linux enterprise server,container_image,keycloak-init:2.3.0-dev-51,keycloak-init.json,SUSE-SU-2022:1149-1,libsoftokn3,3.68.2-3.64.2,3.68.3-150000.3.67.1,suse-cvrf,Security update for mozilla-nss,This update for mozilla-nss fixes the followin...,HIGH,"[https://bugzilla.suse.com/1197903, https://li...",sha256:04fcd3db169b7fe1dfcb69d49bf7aae5ef1de03...,suse-cvrf,SUSE CVRF,https://ftp.suse.com/pub/projects/security/cvrf/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,os-pkgs,suse linux enterprise server,NaN,NaN
4,keycloak-init@sha256:a616d074b14f04fd4a9aebd8b...,keycloak-init:2.3.0-dev-51,15.3,suse linux enterprise server,container_image,keycloak-init:2.3.0-dev-51,keycloak-init.json,SUSE-SU-2022:1149-1,mozilla-nss,3.68.2-3.64.2,3.68.3-150000.3.67.1,suse-cvrf,Security update for mozilla-nss,This update for mozilla-nss fixes the followin...,HIGH,"[https://bugzilla.suse.com/1197903, https://li...",sha256:04fcd3db169b7fe1dfcb69d49bf7aae5ef1de03...,suse-cvrf,SUSE CVRF,https://ftp.suse.com/pub/projects/security/cvrf/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,armdocker.rnd.ericsson.se/proj-mxe-ci-internal...,os-pkgs,suse linux enterprise server,NaN,NaN


### Count of modules by CVE

In [17]:
cve_with_modules_df = modified_base_df[['VulnerabilityID', 'Severity', 'repoTags']].copy().drop_duplicates()
cve_with_modules_df.columns=['cve_id', 'severity', 'image_name']
cve_with_modules_df['image_name']= cve_with_modules_df.groupby(['cve_id', 'severity'])['image_name'].transform(lambda x : '|'.join(x))
cve_with_modules_df['image_count']= cve_with_modules_df.apply(lambda rec: len(rec['image_name'].split('|')), axis=1)
cve_with_modules_df.drop_duplicates(inplace=True)
cve_with_modules_df['rank'] = cve_with_modules_df['severity'].map(SEVERITY_DICT)
cve_with_modules_df.sort_values(by=['rank','image_count'],ascending=[True, True],inplace=True)
cve_with_modules_df = cve_with_modules_df.drop(labels=['rank'],axis=1).reset_index(drop=True)

print(cve_with_modules_df)

                 cve_id  severity                                         image_name  image_count
0        CVE-2022-28391  CRITICAL                                   gui:2.3.0-dev-51            1
1        CVE-2022-26612  CRITICAL                     spark-operator:3.49.0-4-1.1.19            1
2        CVE-2022-21724  CRITICAL               eric-sec-access-mgmt-image:11.0.0-73            1
3        CVE-2021-43816  CRITICAL                            eric-pm-server:9.0.0-27            1
4        CVE-2022-22965  CRITICAL                             eric-ctrl-bro:5.4.0-35            1
5         CVE-2022-0839  CRITICAL  author-service:2.3.0-dev-51|model-catalogue-se...            4
6   SUSE-SU-2022:1164-1      HIGH                       pre-upgrade-job:2.3.0-dev-51            1
7        CVE-2022-23308      HIGH                                   gui:2.3.0-dev-51            1
8        CVE-2021-30560      HIGH                                   gui:2.3.0-dev-51            1
9         CVE-2022-1

In [18]:
cve_with_modules_df.to_html(f"{output_dir}/{REPORT_NAMES['IMAGE_COUNT_BY_CVE']}.html")
cve_with_modules_df.to_csv(f"{output_dir}/{REPORT_NAMES['IMAGE_COUNT_BY_CVE']}.csv", index=False)

## CVEs by Month Year

Reused code written for Anchore.

If LastModifiedDate is Nan, fill with 1900-01-01

In [19]:
cve_by_date_severity_df = modified_base_df[['Severity', 'VulnerabilityID', 'LastModifiedDate']].copy()

cve_by_date_severity_df['LastModifiedDate'].fillna(value='1900-01-01T00:00Z', inplace=True)

cve_by_date_severity_df['LastModifiedDateTime']=pd.to_datetime(cve_by_date_severity_df['LastModifiedDate'])
cve_by_date_severity_df['MonthYear'] = pd.DatetimeIndex(cve_by_date_severity_df['LastModifiedDate']).strftime('%Y-%m')

severity_by_month_year_interim_df = cve_by_date_severity_df.sort_values(by=['LastModifiedDateTime'], ascending=False).drop(['LastModifiedDateTime'], axis=1)

severity_by_month_year_df = severity_by_month_year_interim_df[['MonthYear','VulnerabilityID', 'Severity']].drop_duplicates()

severity_by_month_year_df = severity_by_month_year_df.drop("VulnerabilityID", axis=1).value_counts().reset_index()

severity_by_month_year_df.columns = ['MonthYear', 'Severity', 'CVE_Count']

severity_by_month_year_df

,MonthYear,Severity,CVE_Count
0,2022-04,HIGH,23
1,1900-01,HIGH,8
2,2022-03,HIGH,5
3,2022-04,CRITICAL,5
4,2021-11,HIGH,4
5,2021-07,HIGH,2
6,2018-01,HIGH,1
7,2018-08,HIGH,1
8,2019-05,HIGH,1
9,2020-03,HIGH,1


## Pivot report - Not sorted

In [20]:
pivot_df = severity_by_month_year_df.pivot_table(index='MonthYear', columns='Severity', values='CVE_Count', fill_value=0)

pivot_df

Severity,CRITICAL,HIGH
MonthYear,,
1900-01,0,8
2018-01,0,1
2018-08,0,1
2019-05,0,1
2020-03,0,1
2021-07,0,2
2021-08,0,1
2021-10,0,1
2021-11,0,4


## Pivot Report - Sorted by MonthYear

In [21]:
format_pivot_df = pivot_df.reset_index()

format_pivot_df['MonthDate'] = pd.to_datetime(format_pivot_df['MonthYear'])

format_pivot_df.sort_values(by='MonthDate', ascending=False, inplace=True)

format_pivot_df.set_index(['MonthYear'], inplace=True)

format_pivot_df.drop(['MonthDate'], inplace=True, axis=1)

format_pivot_df

Severity,CRITICAL,HIGH
MonthYear,,
2022-04,5,23
2022-03,1,5
2022-01,0,1
2021-12,0,1
2021-11,0,4
2021-10,0,1
2021-08,0,1
2021-07,0,2
2020-03,0,1


In [22]:
format_pivot_df.to_html(f"{output_dir}/{REPORT_NAMES['CVE_COUNT_BY_SEVERITY_AND_AGE']}.html")
format_pivot_df.to_csv(f"{output_dir}/{REPORT_NAMES['CVE_COUNT_BY_SEVERITY_AND_AGE']}.csv", index=False)

In [23]:

with pd.ExcelWriter(f"{output_dir}/trivy_{REPORT_NAMES['CONSOLIDATED_EXCEL_REPORT']}.xlsx") as excelWriter: 
    detailed_report_df.to_excel(excelWriter, sheet_name=REPORT_NAMES['DETAILED_REPORT'], index=False)
    severity_counts_df.to_excel(excelWriter, sheet_name=REPORT_NAMES['CVE_COUNT_BY_SEVERITY'], index=False)
    cve_with_modules_df.to_excel(excelWriter, sheet_name=REPORT_NAMES['IMAGE_COUNT_BY_CVE'], index=False)
    format_pivot_df.to_excel(excelWriter, sheet_name=REPORT_NAMES['CVE_COUNT_BY_SEVERITY_AND_AGE'], index=True)